# Demo of `secop-ophyd` Integration 




### Bluesky Runenegine Setup

In [1]:
from databroker.v2 import temp
from bluesky import RunEngine, Msg
import bluesky.plan_stubs as bps
from bluesky.plans import count,list_scan
import matplotlib.pyplot as plt
from bluesky.run_engine import get_bluesky_event_loop
from bluesky.callbacks.best_effort import BestEffortCallback
from pprint import pprint


import bluesky.preprocessors as bpp

from secop_ophyd.SECoPDevices import SECoP_Node_Device, SECoPReadableDevice, SECoPMoveableDevice, SECoP_CMD_Device

import random

from bluesky.utils import ProgressBarManager



# Create a run engine and a temporary file backed database. Send all the documents from the RE into that database
RE = RunEngine({},call_returns_result=True)
db = temp()
bec = BestEffortCallback()
RE.subscribe(bec)
RE.waiting_hook = ProgressBarManager()
RE.ignore_callback_exceptions = False
RE.subscribe(db.v1.insert)



#Example of adding metadata to RE environment
investigation_id = "kmnk2n34"

RE.md["investigation_id"] = investigation_id



### SECoP-Ophyd Device generation

In [2]:


# Connect to SEC Node and generate ophyd device tree
cryo_node = await SECoP_Node_Device.create('localhost','10769',RE.loop)


# cryo node is only comprised of a single module: cryo
cryo:SECoPMoveableDevice= cryo_node.cryo




cryo_7.frappy.demo ready


In [3]:
pprint(await cryo.describe())

{'cryo_7-frappy-demo-cryo-value': {'SECOP_datainfo': {'min': 0.0,
                                                      'type': 'double',
                                                      'unit': 'K'},
                                   'SECoP_dtype': 'double',
                                   '_test': 'customized value',
                                   'description': 'regulation temperature',
                                   'dtype': 'number',
                                   'min': 0.0,
                                   'readonly': True,
                                   'shape': [],
                                   'source': 'localhost:10769:cryo_7.frappy.demo:cryo:value',
                                   'unit': 'K'}}


In [6]:
pprint(await cryo.describe_configuration())

{'cryo_7-frappy-demo-cryo-d': {'SECOP_datainfo': {'max': 100.0,
                                                  'min': 0.0,
                                                  'type': 'double'},
                               'SECoP_dtype': 'double',
                               'description': "regulation coefficient 'd'",
                               'dtype': 'number',
                               'group': 'pid',
                               'max': 100.0,
                               'min': 0.0,
                               'readonly': False,
                               'shape': [],
                               'source': 'localhost:10769:cryo_7.frappy.demo:cryo:d'},
 'cryo_7-frappy-demo-cryo-description': {'dtype': 'string',
                                         'shape': [],
                                         'source': 'description'},
 'cryo_7-frappy-demo-cryo-features': {'dtype': 'array',
                                      'shape': [],
                   

In [4]:
#p = RE(count([cryo],10,0.5))

In [5]:
#p = RE(list_scan([cryo],cryo,[11.0,12.0]))
p = RE(bps.mv(cryo,11))